## Prediksi NO2

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from datetime import timedelta
import numpy as np

# === Baca dataset ===
df = pd.read_csv("Data/NO2_Surabaya.csv")

# Jika header ternyata terbaca sebagai string tunggal (kadang Excel menulis ulang header),
# bisa pakai fallback ini:
if df.columns[0] == "Tanggal,NO2":
    df = pd.read_csv("Data/NO2_Surabaya.csv", names=["Tanggal", "NO2"], skiprows=1)

# Ubah tipe data
df["Tanggal"] = pd.to_datetime(df["Tanggal"])
df["NO2"] = pd.to_numeric(df["NO2"], errors="coerce")

# Hapus baris kosong atau NaN
df = df.dropna(subset=["Tanggal", "NO2"])

# Urutkan berdasarkan tanggal
df = df.sort_values("Tanggal").reset_index(drop=True)

# Cek hasil
print(df.head())
print(df.info())


     Tanggal       NO2
0 2023-10-25  0.000026
1 2023-10-26  0.000066
2 2023-10-27  0.000048
3 2023-10-29  0.000052
4 2023-10-30  0.000055
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Tanggal  301 non-null    datetime64[ns]
 1   NO2      301 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 4.8 KB
None


In [6]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plt.figure(figsize=(12,6))
plt.plot(df['Tanggal'], df['NO2'], marker='o', linestyle='-', color='blue')
plt.title("Konsentrasi NO₂ Harian di Surabaya", fontsize=14)
plt.xlabel("Tanggal")
plt.ylabel("Konsentrasi NO₂")
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot autokorelasi
fig, axes = plt.subplots(1, 2, figsize=(12,4))
plot_acf(df['NO2'], ax=axes[0], lags=20)
plot_pacf(df['NO2'], ax=axes[1], lags=20)
axes[0].set_title("ACF (Autocorrelation)")
axes[1].set_title("PACF (Partial Autocorrelation)")
plt.tight_layout()
plt.show()


In [7]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(df['NO2'].dropna())
print("ADF Statistic:", result[0])
print("p-value:", result[1])
print("Critical Values:", result[4])

if result[1] > 0.05:
    print("⚠️ Data tidak stasioner → perlu differencing.")
    df['NO2_diff'] = df['NO2'].diff().dropna()
else:
    print("✅ Data sudah stasioner.")


ADF Statistic: -4.268932099199984
p-value: 0.0005025789315839579
Critical Values: {'1%': np.float64(-3.452789844280995), '5%': np.float64(-2.871421512222641), '10%': np.float64(-2.5720351510944512)}
✅ Data sudah stasioner.


In [8]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Bangun model SARIMAX
model = SARIMAX(df['NO2'], order=(1,1,1), seasonal_order=(1,1,1,7))
results = model.fit(disp=False)

print(results.summary())

# Prediksi 7 hari ke depan
future_steps = 7
forecast = results.get_forecast(steps=future_steps)
pred_mean = forecast.predicted_mean
pred_ci = forecast.conf_int()

# Buat tanggal prediksi
last_date = df['Tanggal'].iloc[-1]
future_dates = [last_date + timedelta(days=i) for i in range(1, future_steps+1)]

pred_df = pd.DataFrame({
    'Tanggal': future_dates,
    'Prediksi_NO2': pred_mean.values,
    'Lower_CI': pred_ci.iloc[:, 0].values,
    'Upper_CI': pred_ci.iloc[:, 1].values
})

print(pred_df)


/usr/local/python/3.12.1/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                     SARIMAX Results                                     
Dep. Variable:                               NO2   No. Observations:                  301
Model:             SARIMAX(1, 1, 1)x(1, 1, 1, 7)   Log Likelihood                2696.906
Date:                           Mon, 03 Nov 2025   AIC                          -5383.813
Time:                                   02:54:51   BIC                          -5365.412
Sample:                                        0   HQIC                         -5376.443
                                           - 301                                         
Covariance Type:                             opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0246   2.93e-20   8.37e+17      0.000       0.025       0.025
ma.L1         -0.7488   2.57e-20  -2.92e+19

In [9]:
plt.figure(figsize=(12,6))
plt.plot(df['Tanggal'], df['NO2'], label='Data Aktual', color='blue')
plt.plot(pred_df['Tanggal'], pred_df['Prediksi_NO2'], color='red', marker='o', label='Prediksi 7 Hari ke Depan')
plt.fill_between(pred_df['Tanggal'], pred_df['Lower_CI'], pred_df['Upper_CI'], color='pink', alpha=0.3)
plt.title("Prediksi Konsentrasi NO₂ 7 Hari ke Depan (SARIMAX)", fontsize=14)
plt.xlabel("Tanggal")
plt.ylabel("Konsentrasi NO₂")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


In [11]:
import folium

# Koordinat Surabaya
lat, lon = -7.2575, 112.7521

# Nilai rata-rata prediksi
mean_pred = pred_df['Prediksi_NO2'].mean()

# Fungsi warna sesuai konsentrasi
def color_scale(val):
    if val < 2.5e-5:
        return 'green'
    elif val < 5e-5:
        return 'orange'
    else:
        return 'red'

m = folium.Map(location=[lat, lon], zoom_start=11)
folium.CircleMarker(
    location=[lat, lon],
    radius=15,
    color=color_scale(mean_pred),
    fill=True,
    fill_opacity=0.6,
    popup=f"Prediksi rata-rata NO₂ (7 hari): {mean_pred:.2e}"
).add_to(m)

m.save("map_no2.html")
m

In [12]:
# Cell 7: Generate app.py untuk Streamlit
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from datetime import timedelta
import folium
from streamlit_folium import st_folium

st.title("🌫️ Prediksi Konsentrasi NO₂ Surabaya (7 Hari ke Depan)")

uploaded_file = st.file_uploader("Upload file CSV NO₂", type="csv")

if uploaded_file:
    df = pd.read_csv(uploaded_file)
    if df.columns[0] == "Tanggal,NO2":
        df = pd.read_csv(uploaded_file, names=["Tanggal", "NO2"], skiprows=1)
    df['Tanggal'] = pd.to_datetime(df['Tanggal'])
    df['NO2'] = pd.to_numeric(df['NO2'], errors='coerce')
    df = df.dropna(subset=['Tanggal','NO2']).sort_values('Tanggal')

    st.line_chart(df.set_index('Tanggal')['NO2'])

    model = SARIMAX(df['NO2'], order=(1,1,1), seasonal_order=(1,1,1,7))
    results = model.fit(disp=False)
    forecast = results.get_forecast(steps=7)
    pred_mean = forecast.predicted_mean
    future_dates = [df['Tanggal'].iloc[-1] + timedelta(days=i) for i in range(1,8)]
    pred_df = pd.DataFrame({'Tanggal': future_dates, 'Prediksi_NO2': pred_mean.values})
    
    st.subheader("📅 Prediksi 7 Hari ke Depan")
    st.dataframe(pred_df)

    plt.figure(figsize=(10,5))
    plt.plot(df['Tanggal'], df['NO2'], label='Aktual', color='blue')
    plt.plot(pred_df['Tanggal'], pred_df['Prediksi_NO2'], color='red', marker='o', label='Prediksi')
    plt.legend()
    st.pyplot(plt)

    lat, lon = -7.2575, 112.7521
    m = folium.Map(location=[lat, lon], zoom_start=11)
    mean_pred = pred_df['Prediksi_NO2'].mean()
    folium.CircleMarker([lat, lon], radius=15, color='red', fill=True,
                        popup=f"Prediksi rata-rata NO₂: {mean_pred:.2e}").add_to(m)
    st_folium(m, width=700)
""")
print("✅ File app.py siap dijalankan di Streamlit!")


✅ File app.py siap dijalankan di Streamlit!
